In [51]:
import geopandas as gpd
import pandas as pd
import numpy as np

## Configuraciones Iniciales
ocupacion_seccion_2011_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Secciones\Ocupados de 16 y más años_Ocupacion_Secciones_Censo_2011.csv"
ocupacion_seccion_2021_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Secciones\Ocupados de 16 y más años_Ocupacion_Secciones_Censo_2021.csv"
cartografia_seccion_shp_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Shapefiles\da_cartografiaSeccionCensal-25830\da_cartografiaSeccionCensalPolygon.shp"

## Lectura inicial de datos ocupacion
df_ocupacion_seccion_2011 =  pd.read_csv(ocupacion_seccion_2011_csv_path,sep=";")
df_ocupacion_seccion_2021 =  pd.read_csv(ocupacion_seccion_2021_csv_path,sep=";")
df_cartografia_seccion = gpd.read_file(cartografia_seccion_shp_path)

# Ver las columnas
#print(df_ocupacion_seccion_2011.columns)
#print(df_ocupacion_seccion_2021.columns)
print(df_cartografia_seccion.columns)

# Inspeccionar los primeros registros
#print(df_ocupacion_seccion_2011.head())
#print(df_ocupacion_seccion_2021.head())
#print(df_cartografia_seccion.head())

#Se realiza una normalizacion de los csv de ocupacion en dos columnas ocupacion y personas 
df_ocupacion_seccion_2011_long = pd.melt(df_ocupacion_seccion_2011, id_vars=["Municipio de residencia","Seccion de residencia"], var_name='Ocupacion', value_name='Personas')
#print(df_ocupacion_seccion_2011_long.columns)
#print(df_ocupacion_seccion_2011_long.head())
df_ocupacion_seccion_2021_long = pd.melt(df_ocupacion_seccion_2021, id_vars=["Municipio de residencia","Sección "], var_name='Ocupacion', value_name='Personas')
#print(df_ocupacion_seccion_2021_long.columns)
#print(df_ocupacion_seccion_2021_long.head())

#Cambio el nombre de la columna 'Sección ' a Seccion de residencia
df_ocupacion_seccion_2021_long.rename(columns={'Sección ': 'Seccion de residencia'}, inplace=True)
#print(df_ocupacion_seccion_2021_long.columns)

## Eliminar filas ##
#Eliminar filas de municipio Total
df_ocupacion_seccion_2011_long = df_ocupacion_seccion_2011_long[df_ocupacion_seccion_2011_long['Municipio de residencia'] != 'Total']
df_ocupacion_seccion_2021_long = df_ocupacion_seccion_2021_long[df_ocupacion_seccion_2021_long['Municipio de residencia'] != 'Total']
#Eliminamos las filas de ocupaciones militares, no consta y totales y tambien seccion de residencia Total
df_ocupacion_seccion_2011_long = df_ocupacion_seccion_2011_long[df_ocupacion_seccion_2011_long['Ocupacion'] != '0 - Ocupaciones militares']
df_ocupacion_seccion_2021_long = df_ocupacion_seccion_2021_long[df_ocupacion_seccion_2021_long['Ocupacion'] != '0 - Ocupaciones militares']
df_ocupacion_seccion_2011_long = df_ocupacion_seccion_2011_long[df_ocupacion_seccion_2011_long['Ocupacion'] != 'No consta']
df_ocupacion_seccion_2021_long = df_ocupacion_seccion_2021_long[df_ocupacion_seccion_2021_long['Ocupacion'] != 'No consta']
df_ocupacion_seccion_2011_long = df_ocupacion_seccion_2011_long[df_ocupacion_seccion_2011_long['Ocupacion'] != 'Total']
df_ocupacion_seccion_2021_long = df_ocupacion_seccion_2021_long[df_ocupacion_seccion_2021_long['Ocupacion'] != 'Total']
df_ocupacion_seccion_2011_long = df_ocupacion_seccion_2011_long[df_ocupacion_seccion_2011_long['Seccion de residencia'] != 'Total']
df_ocupacion_seccion_2021_long = df_ocupacion_seccion_2021_long[df_ocupacion_seccion_2021_long['Seccion de residencia'] != 'Total']
#print(df_ocupacion_seccion_2011_long.head())
#print(df_ocupacion_seccion_2021_long.head())

#Contamos el numero de valores que hay en estos casos
num_filas_en_blanco_2011 = (df_ocupacion_seccion_2011_long['Personas'].isna()).sum()
num_filas_con_asterisco_2011 = (df_ocupacion_seccion_2011_long['Personas'] == "*").sum()
num_filas_contiene_asterisco_2011 = (df_ocupacion_seccion_2011_long['Personas'].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2011)#0
#print(num_filas_con_asterisco_2011)#2905
#print(num_filas_contiene_asterisco_2011)#2905
num_filas_en_blanco_2021 = (df_ocupacion_seccion_2021_long['Personas'].isna()).sum()
num_filas_con_asterisco_2021 = (df_ocupacion_seccion_2021_long['Personas'] == "*").sum()
num_filas_contiene_asterisco_2021 = (df_ocupacion_seccion_2021_long['Personas'].apply(str).str.contains(r'\*')).sum()
#print(num_filas_en_blanco_2021)#0
#print(num_filas_con_asterisco_2021)#0
#print(num_filas_contiene_asterisco_2021)#0

## Añadir columnas ##
#Añadir columna ID_SECCENS con el ID de la seccion
df_ocupacion_seccion_2011_long["NUMSECCENS"] = df_ocupacion_seccion_2011_long["Seccion de residencia"].str[-5:].astype(float)
df_ocupacion_seccion_2021_long["NUMSECCENS"] = df_ocupacion_seccion_2021_long["Seccion de residencia"].str[-5:].astype(float)
#print(df_ocupacion_seccion_2011_long.columns)
#print(df_ocupacion_seccion_2021_long.columns)
#Añadir columna Id_Ocupacion
#Se puede hacer asi porque solo va de 1 a 9
df_ocupacion_seccion_2011_long["Id_Ocupacion"] = df_ocupacion_seccion_2011_long["Ocupacion"].str[0].astype(float)
df_ocupacion_seccion_2021_long["Id_Ocupacion"] = df_ocupacion_seccion_2021_long["Ocupacion"].str[0].astype(float)

## Gestion de Nulos y valores extraños ##
#Sustituimos los valores * por NA
df_ocupacion_seccion_2011_long['Personas'] = df_ocupacion_seccion_2011_long['Personas'].replace('*', np.nan)
df_ocupacion_seccion_2021_long['Personas'] = df_ocupacion_seccion_2021_long['Personas'].replace('*', np.nan)
#Columna Personas como float
df_ocupacion_seccion_2011_long['Personas'] = df_ocupacion_seccion_2011_long['Personas'].astype(float)
df_ocupacion_seccion_2021_long['Personas'] = df_ocupacion_seccion_2021_long['Personas'].astype(float)
#Contamos el numero de valores tras los nuevos ajustes
num_filas_en_blanco_2021 = (df_ocupacion_seccion_2011_long['Personas'].isna()).sum()
num_filas_con_asterisco_2021 = (df_ocupacion_seccion_2021_long['Personas'] == "*").sum()
#print(num_filas_en_blanco_2021)#2905
#print(num_filas_con_asterisco_2021)#0

## Calcular % Personas ocupacion por seccion con valor medio 3 ##
#Los valores con * tienen un valor entre 1-5 según el INE. Utilizaremos la media (3) como estimacion
# Calcular la desviación estándar antes de la imputación
desviacion_imputada_2011= df_ocupacion_seccion_2011_long.groupby('Seccion de residencia')['Personas'].std()
#print("Desviación estándar por distrito antes de imputar 3:", desviacion_imputada_2011)
#No obstante vamos a ver si la desviacion del dato cambia mucho respecto a lo anterior usando el 3
df_ocupacion_seccion_2011_long['Personas'] = df_ocupacion_seccion_2011_long['Personas'].replace(np.nan, 3)
df_ocupacion_seccion_2021_long['Personas'] = df_ocupacion_seccion_2021_long['Personas'].replace(np.nan, 3)
desviacion_imputada_2011= df_ocupacion_seccion_2011_long.groupby('Seccion de residencia')['Personas'].std()
#print("Desviación estándar por distrito despues de imputar 3:", desviacion_imputada_2011)

#Calcular el total de personas por seccion
df_ocupacion_seccion_2011_long['Total_personas_seccion'] = df_ocupacion_seccion_2011_long.groupby('Seccion de residencia')['Personas'].transform('sum')
df_ocupacion_seccion_2021_long['Total_personas_seccion'] = df_ocupacion_seccion_2021_long.groupby('Seccion de residencia')['Personas'].transform('sum')
#Calcular porcentaje sobre el total
df_ocupacion_seccion_2011_long['Personas %'] = (df_ocupacion_seccion_2011_long['Personas'] / df_ocupacion_seccion_2011_long['Total_personas_seccion'])
df_ocupacion_seccion_2021_long['Personas %'] = (df_ocupacion_seccion_2021_long['Personas'] / df_ocupacion_seccion_2021_long['Total_personas_seccion'])
print(df_ocupacion_seccion_2011_long[df_ocupacion_seccion_2011_long['NUMSECCENS'] == 1001])
#print(df_ocupacion_seccion_2011_long[df_ocupacion_seccion_2011_long['NUMSECCENS'] == 3001])
#print(df_ocupacion_seccion_2011_long[df_ocupacion_seccion_2011_long['NUMSECCENS'] == 5001])
#print(df_ocupacion_seccion_2021_long[df_ocupacion_seccion_2021_long['NUMSECCENS'] == 1001])


Index(['ID_SECCENS', 'NUMSECCENS', 'FECALTA', 'FECBAJA', 'geometry'], dtype='object')
     Municipio de residencia Seccion de residencia  \
428             29067 Málaga    Sección 2906701001   
856             29067 Málaga    Sección 2906701001   
1284            29067 Málaga    Sección 2906701001   
1712            29067 Málaga    Sección 2906701001   
2140            29067 Málaga    Sección 2906701001   
2568            29067 Málaga    Sección 2906701001   
2996            29067 Málaga    Sección 2906701001   
3424            29067 Málaga    Sección 2906701001   
3852            29067 Málaga    Sección 2906701001   

                                              Ocupacion  Personas  NUMSECCENS  \
428                           1 - Directores y gerentes       3.0      1001.0   
856   2 - Técnicos y profesionales científicos e int...     225.0      1001.0   
1284               3 - Técnicos; profesionales de apoyo       3.0      1001.0   
1712  4 - Empleados contables, administrativos y 

In [53]:
#Export trabajos 'top'
#Los trabajos top son Directores y gerentes; Técnicos y profesionales científicos e intelectuales; Técnicos; profesionales de apoyo
df_ocupacion_distrito_2011_long_only_knoledge = df_ocupacion_seccion_2011_long[df_ocupacion_seccion_2011_long['Id_Ocupacion'].isin([1, 2, 3])]
df_ocupacion_distrito_2021_long_only_knoledge = df_ocupacion_seccion_2021_long[df_ocupacion_seccion_2021_long['Id_Ocupacion'].isin([1, 2, 3])]

# Agrupar por distrito y sumar 'Personas' y 'Porcentaje'
df_agrupado_2011 = df_ocupacion_distrito_2011_long_only_knoledge.groupby(['NUMSECCENS','Seccion de residencia'], as_index=False).agg({
    'Personas %': 'sum'
})

df_agrupado_2021 = df_ocupacion_distrito_2021_long_only_knoledge.groupby(['NUMSECCENS','Seccion de residencia'], as_index=False).agg({
    'Personas %': 'sum'
})

#Renombrar columna
df_agrupado_2011.rename(columns={'Personas %': 'kw_pct'}, inplace=True)
df_agrupado_2021.rename(columns={'Personas %': 'kw_pct'}, inplace=True)

print(df_agrupado_2011)
print(df_agrupado_2021)

df_agrupado_2011.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Secciones\2011_Secciones_Ocupacion_V2.csv",index=False,sep=";")
df_agrupado_2021.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Secciones\2021_Secciones_Ocupacion_V2.csv",index=False,sep=";")


     NUMSECCENS Seccion de residencia    kw_pct
0        1001.0    Sección 2906701001  0.410302
1        1002.0    Sección 2906701002  0.899441
2        1003.0    Sección 2906701003  0.924686
3        1004.0    Sección 2906701004  0.305556
4        2001.0    Sección 2906702001  0.333333
..          ...                   ...       ...
423     11006.0    Sección 2906711006  0.013294
424     11007.0    Sección 2906711007  0.362579
425     11008.0    Sección 2906711008  0.393258
426     11009.0    Sección 2906711009  0.043062
427     11010.0    Sección 2906711010  0.185366

[428 rows x 3 columns]
     NUMSECCENS Seccion de residencia    kw_pct
0        1001.0    Sección 2906701001  0.576744
1        2003.0    Sección 2906702003  0.454545
2        2004.0    Sección 2906702004  0.752809
3        2009.0    Sección 2906702009  0.359223
4        2013.0    Sección 2906702013  0.698925
..          ...                   ...       ...
131     11006.0    Sección 2906711006  0.173913
132     11007.0 

In [55]:
#Export variables
df_seccion_2011_long_export3  = df_ocupacion_seccion_2011_long.pivot(index='NUMSECCENS', columns='Ocupacion', values='Personas %')
df_seccion_2021_long_export3  = df_ocupacion_seccion_2021_long.pivot(index='NUMSECCENS', columns='Ocupacion', values='Personas %')

print(df_seccion_2011_long_export3)
print(df_seccion_2021_long_export3)

df_seccion_2011_long_export3.to_csv(r'C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Secciones\2011_Secciones_Ocupacion_V3.csv',index=True,sep=";")
df_seccion_2021_long_export3.to_csv(r'C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Secciones\2021_Secciones_Ocupacion_V3.csv',index=True,sep=";")

Ocupacion   1 - Directores y gerentes  \
NUMSECCENS                              
1001.0                       0.005329   
1002.0                       0.016760   
1003.0                       0.012552   
1004.0                       0.013889   
2001.0                       0.111111   
...                               ...   
11006.0                      0.004431   
11007.0                      0.003171   
11008.0                      0.095073   
11009.0                      0.014354   
11010.0                      0.007317   

Ocupacion   2 - Técnicos y profesionales científicos e intelectuales  \
NUMSECCENS                                                             
1001.0                                               0.399645          
1002.0                                               0.865922          
1003.0                                               0.899582          
1004.0                                               0.277778          
2001.0                            